In [ ]:
from pathlib import Path
from asim.dataset.scene.arrow_scene import ArrowScene
from asim.common.visualization.matplotlib.plots import plot_scene_at_iteration



log_name = "2021.06.07.12.54.00_veh-35_01843_02314"
log_file = Path(f"/home/daniel/asim_workspace/data/nuplan_mini_val/{log_name}.arrow")
scene = ArrowScene(log_file)

plot_scene_at_iteration(scene, iteration=10)

In [ ]:
from matplotlib import pyplot as plt
from asim.common.visualization.matplotlib.plots import _plot_scene_on_ax

fig, ax = plt.subplots(figsize=(10, 10))
_plot_scene_on_ax(ax, scene)



box_detections = scene.get_box_detections_at_iteration(0)

ego_polygon = scene.get_ego_vehicle_state_at_iteration(0).bounding_box.shapely_polygon
polygons = [ego_polygon] + [box.bounding_box_se2.shapely_polygon for box in box_detections]
from shapely.strtree import STRtree


tree = STRtree(polygons)

In [ ]:
indices, distances = tree.query_nearest([ego_polygon,ego_polygon], exclusive=True, return_distance=True)

# ego_polygon
distances